In [69]:
# !pip install prophet
# !pip install joblib

In [70]:
import pandas as pd
from prophet import Prophet
import joblib  # For saving models
import os


In [71]:
df = pd.read_csv('sensor_data_2024_to_2025.csv')

In [72]:
df.head()

,Timestamp,Temperature (°C),Humidity (%),MQ2 (%),MQ5 (%),MQ135 (%)
0,1/1/2024 0:00,23.4,61.8,31.8,58.2,49.5
1,1/1/2024 1:00,28.6,68.1,8.0,24.8,20.8
2,1/1/2024 2:00,34.8,65.4,36.2,11.7,31.8
3,1/1/2024 3:00,29.7,43.9,49.6,12.3,13.0
4,1/1/2024 4:00,20.8,45.1,20.1,14.7,28.3


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9528 entries, 0 to 9527
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Timestamp         9192 non-null   object 
 1   Temperature (°C)  9192 non-null   float64
 2   Humidity (%)      9192 non-null   float64
 3   MQ2 (%)           9192 non-null   float64
 4   MQ5 (%)           9192 non-null   float64
 5   MQ135 (%)         9192 non-null   float64
dtypes: float64(5), object(1)
memory usage: 446.8+ KB


In [74]:
df.shape

(9528, 6)

In [75]:
df.notna().sum()

Timestamp           9192
Temperature (°C)    9192
Humidity (%)        9192
MQ2 (%)             9192
MQ5 (%)             9192
MQ135 (%)           9192
dtype: int64

In [76]:
# Assuming df is your DataFrame
df = df.dropna()

# Convert 'Timestamp' to datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%m/%d/%Y %H:%M')

# List of columns to forecast
target_columns = ['Temperature (°C)', 'Humidity (%)', 'MQ2 (%)', 'MQ5 (%)', 'MQ135 (%)']

In [77]:
# Directory to save models
model_dir = 'prophet_models'
os.makedirs(model_dir, exist_ok=True)  # Create directory if it doesn't exist

# Dictionary to store forecasts
forecasts = {}

# Loop through each target variable
for column in target_columns:
    print(f"Forecasting {column}...")
    
    # Prepare the data for Prophet
    df_prophet = df[['Timestamp', column]].rename(columns={'Timestamp': 'ds', column: 'y'})
    
    # Initialize and fit the Prophet model
    model = Prophet()
    model.fit(df_prophet)
    
    # Create future DataFrame for predictions
    future = model.make_future_dataframe(periods=365)  # Forecast 365 days into the future
    
    # Make predictions
    forecast = model.predict(future)
    
    # Store the forecast in the dictionary
    forecasts[column] = forecast
    
    # Save the model to disk
    model_filename = os.path.join(model_dir, f'{column}_prophet_model.joblib')
    joblib.dump(model, model_filename)
    print(f"Model for {column} saved to {model_filename}")

# Access forecasts for each variable
for column, forecast in forecasts.items():
    print(f"Forecast for {column}:")
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

Forecasting Temperature (°C)...


02:22:06 - cmdstanpy - INFO - Chain [1] start processing
02:22:08 - cmdstanpy - INFO - Chain [1] done processing


Model for Temperature (°C) saved to prophet_models\Temperature (°C)_prophet_model.joblib
Forecasting Humidity (%)...


02:22:11 - cmdstanpy - INFO - Chain [1] start processing
02:22:14 - cmdstanpy - INFO - Chain [1] done processing


Model for Humidity (%) saved to prophet_models\Humidity (%)_prophet_model.joblib
Forecasting MQ2 (%)...


...

Model for MQ2 (%) saved to prophet_models\MQ2 (%)_prophet_model.joblib
Forecasting MQ5 (%)...


02:22:23 - cmdstanpy - INFO - Chain [1] start processing
02:22:26 - cmdstanpy - INFO - Chain [1] done processing


Model for MQ5 (%) saved to prophet_models\MQ5 (%)_prophet_model.joblib
Forecasting MQ135 (%)...


02:22:29 - cmdstanpy - INFO - Chain [1] start processing
02:22:33 - cmdstanpy - INFO - Chain [1] done processing


Model for MQ135 (%) saved to prophet_models\MQ135 (%)_prophet_model.joblib
Forecast for Temperature (°C):
                      ds       yhat  yhat_lower  yhat_upper
9552 2026-01-13 23:00:00  26.867307   21.747031   32.340268
9553 2026-01-14 23:00:00  26.805757   21.455139   32.257948
9554 2026-01-15 23:00:00  26.734647   21.168588   32.255239
9555 2026-01-16 23:00:00  26.772155   21.383023   32.385523
9556 2026-01-17 23:00:00  26.738861   21.397895   32.339098
Forecast for Humidity (%):
                      ds       yhat  yhat_lower  yhat_upper
9552 2026-01-13 23:00:00  51.510254   36.368326   66.842104
9553 2026-01-14 23:00:00  51.404375   37.307152   65.787237
9554 2026-01-15 23:00:00  52.014244   37.392003   66.297599
9555 2026-01-16 23:00:00  51.286697   37.494881   66.381614
9556 2026-01-17 23:00:00  51.953000   38.043844   67.665547
Forecast for MQ2 (%):
                      ds       yhat  yhat_lower  yhat_upper
9552 2026-01-13 23:00:00  25.732294    7.077913   43.873219
9553 